In [1]:
import os

In [2]:
list_file = list(os.walk('./model'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
list_file.sort()
    
len(list_file)

2495

In [3]:
import pandas as pd

In [4]:
input_ = pd.read_excel('./make.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_

,Part_Code,Make,Make_Code,Model_Url,Code
0,1000392,VW,9992,https://www.atp-autoteile.de/de/product/vehicl...,1000392;9992
1,1016832,Audi,9950,https://www.atp-autoteile.de/de/product/vehicl...,1016832;9950
2,1016832,Seat,9983,https://www.atp-autoteile.de/de/product/vehicl...,1016832;9983
3,1016832,Skoda,9984,https://www.atp-autoteile.de/de/product/vehicl...,1016832;9984
4,1016832,VW,9992,https://www.atp-autoteile.de/de/product/vehicl...,1016832;9992
...,...,...,...,...,...
2490,989599,Peugeot,9978,https://www.atp-autoteile.de/de/product/vehicl...,989599;9978
2491,989696,BMW,9953,https://www.atp-autoteile.de/de/product/vehicl...,989696;9953
2492,98999,Hyundai,10004,https://www.atp-autoteile.de/de/product/vehicl...,98999;10004
2493,98999,KIA,10005,https://www.atp-autoteile.de/de/product/vehicl...,98999;10005


In [5]:
from tqdm import tqdm
from bs4 import BeautifulSoup
from lxml import etree
import json

In [6]:
dict_vehicle = {}
for file in tqdm(list_file, desc='Progress', ncols=77):
    part_code = file.split(';')[0]
    if part_code not in dict_vehicle:
        dict_vehicle[part_code] = []
    
    make = input_.loc[input_['Code'].tolist().index(file.split('.')[0]), 'Make']
    
    with open('./model/' + file, 'r', encoding='UTF-8') as f:
        html = f.read()

        # = = = = = = = = = = = = = = =
        
        soup = BeautifulSoup(html, 'lxml')
        
        # = = = = = = = = = = = = = = =
        
        html = etree.HTML(str(soup))
        
        list_td = html.xpath('//td')
        
        # = = = = = = = = = = = = = = =
        
        for td in list_td:
            model = td.xpath('./span[1]/text()')[0].strip()

            # = = = = = = = = = = = = = = =
        
            engine = td.xpath('./span[2]/text()')[0].strip()[:-1].strip()
        
            # = = = = = = = = = = = = = = =
        
            year = td.xpath('./span[4]/text()')[0].strip()
            
            # = = = = = = = = = = = = = = =
            
            if '-' in year:
                dict_vehicle[part_code].append({'make': make, 'model':model, 'engine':engine, 'year':int(year.split('-')[0].split('/')[1].strip())})
                dict_vehicle[part_code].append({'make': make, 'model':model, 'engine':engine, 'year':int(year.split('-')[1].split('/')[1][:-1].strip())})
            else:
                dict_vehicle[part_code].append({'make': make, 'model':model, 'engine':engine, 'year':int(year.split('/')[1][:-1].strip())})
                
len(dict_vehicle)

Progress: 100%|██████████████████████████| 2495/2495 [00:34<00:00, 73.06it/s]


1366

In [7]:
output = pd.DataFrame()

output

""


In [8]:
for part_code in dict_vehicle:
    part_dict_vehicle_1 = {}
    for vehicle in dict_vehicle[part_code]:
        if vehicle['make'] not in part_dict_vehicle_1:
            part_dict_vehicle_1[vehicle['make']] = {vehicle['model']: [vehicle['year']]}
        else:
            if vehicle['model'] not in part_dict_vehicle_1[vehicle['make']]:
                part_dict_vehicle_1[vehicle['make']][vehicle['model']] = [vehicle['year']]
            else:
                part_dict_vehicle_1[vehicle['make']][vehicle['model']].append(vehicle['year'])
    
    part_dict_vehicle_2 = {}
    for vehicle in dict_vehicle[part_code]:
        if vehicle['make'] not in part_dict_vehicle_2:
            part_dict_vehicle_2[vehicle['make']] = {vehicle['model']: {vehicle['engine']: [vehicle['year']]}}
        else:
            if vehicle['model'] not in part_dict_vehicle_2[vehicle['make']]:
                part_dict_vehicle_2[vehicle['make']][vehicle['model']] = {vehicle['engine']: [vehicle['year']]}
            else:
                if vehicle['engine'] not in part_dict_vehicle_2[vehicle['make']][vehicle['model']]:
                    part_dict_vehicle_2[vehicle['make']][vehicle['model']][vehicle['engine']] = [vehicle['year']]
                else:
                    part_dict_vehicle_2[vehicle['make']][vehicle['model']][vehicle['engine']].append(vehicle['year'])

    part_list_vehicle_1 = []
    for make in part_dict_vehicle_1:
        for model in part_dict_vehicle_1[make]:
            year_max = max(part_dict_vehicle_1[make][model])
            year_min = min(part_dict_vehicle_1[make][model])
            if year_max == year_min:
                part_list_vehicle_1.append(make + ' ' + model + ' ' + str(year_min))
            else:
                part_list_vehicle_1.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
    part_list_vehicle_1.sort()
    vehicle_1 = '\n'.join(part_list_vehicle_1)

    part_list_vehicle_2 = []
    for make in part_dict_vehicle_2:
        for model in part_dict_vehicle_2[make]:
            for engine in part_dict_vehicle_2[make][model]:
                year_max = max(part_dict_vehicle_2[make][model][engine])
                year_min = min(part_dict_vehicle_2[make][model][engine])
                if year_max == year_min:
                    part_list_vehicle_2.append(make + ' ' + model + ' ' + str(year_min) + ' - ' + engine)
                else:
                    part_list_vehicle_2.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' - ' + engine)
    part_list_vehicle_2.sort()
    vehicle_2 = '\n'.join(part_list_vehicle_2)

    df_temp = pd.DataFrame([{'Part_Code': part_code,
                             'Vehicle_1': vehicle_1,
                             'Vehicle_2': vehicle_2}])

    output = pd.concat([output, df_temp], ignore_index=True).fillna('')

output = output.drop_duplicates(ignore_index=True)
output = output.sort_values(by=['Part_Code'], ascending=[True]).reset_index(drop=True)
output.to_excel('./model.xlsx', index=False)

output

,Part_Code,Vehicle_1,Vehicle_2
0,1000392,"VW TRANSPORTER T4 Bus (70B, 70C, 7DB, 7DK, 70J...","VW TRANSPORTER T4 Bus (70B, 70C, 7DB, 7DK, 70J..."
1,1016832,"Audi A3 (8V1, 8VK) 2012-2017\nAudi A3 Cabriole...","Audi A3 (8V1, 8VK) 2012-2016 - 1.4 TFSI\nAudi ..."
2,1019817,"Audi A3 (8V1, 8VK) 2012-2016\nAudi A3 Cabriole...","Audi A3 (8V1, 8VK) 2012 - 2.0 TDI quattro\nAud..."
3,1020670,"Audi A3 (8V1, 8VK) 2012-2016\nAudi A3 Cabriole...","Audi A3 (8V1, 8VK) 2012 - 2.0 TDI quattro\nAud..."
4,103013,"Mercedes-Benz SPRINTER 4,6-t Pritsche/Fahrgest...","Mercedes-Benz SPRINTER 4,6-t Pritsche/Fahrgest..."
...,...,...,...
1361,989187,Mercedes-Benz CLK (C208) 1997-2002\nMercedes-B...,Mercedes-Benz CLK (C208) 1997-2002 - 320\nMerc...
1362,989599,"Citroën C4 AIRCROSS 2012\nMazda 6 Kombi (GJ, G...",Citroën C4 AIRCROSS 2012 - 1.6\nCitroën C4 AIR...
1363,989696,BMW 1 (E81) 2006-2011\nBMW 1 (E87) 2004-2011\n...,BMW 1 (E81) 2006-2011 - 118 d\nBMW 1 (E81) 200...
1364,98999,"Hyundai TUCSON (TL, TLE) 2015\nHyundai TUCSON ...","Hyundai TUCSON (TL, TLE) 2015 - 1.6 GDi\nHyund..."
